### Recursive Solution

In [ ]:
def print_pegs(pegs):
    print(f"A: {pegs['A']}")
    print(f"B: {pegs['B']}")
    print(f"C: {pegs['C']}")
    print("-" * 20)

def hanoi(n, source, auxiliary, destination, pegs):
    if n == 1:
        # Move top disk
        disk = pegs[source].pop()
        pegs[destination].append(disk)
        print(f"Move disk {disk} from {source} to {destination}")
        print_pegs(pegs)
        return

    hanoi(n - 1, source, destination, auxiliary, pegs)

    disk = pegs[source].pop()
    pegs[destination].append(disk)
    print(f"Move disk {disk} from {source} to {destination}")
    print_pegs(pegs)

    hanoi(n - 1, auxiliary, source, destination, pegs)

# Example with 3 disks
n = 3
pegs = {
    "A": list(range(n, 0, -1)),  # Largest at bottom
    "B": [],
    "C": []
}

print("Initial State:")
print_pegs(pegs)
hanoi(n, "A", "B", "C", pegs)


Initial State:
A: [3, 2, 1]
B: []
C: []
--------------------
Move disk 1 from A to C
A: [3, 2]
B: []
C: [1]
--------------------
Move disk 2 from A to B
A: [3]
B: [2]
C: [1]
--------------------
Move disk 1 from C to B
A: [3]
B: [2, 1]
C: []
--------------------
Move disk 3 from A to C
A: []
B: [2, 1]
C: [3]
--------------------
Move disk 1 from B to A
A: [1]
B: [2]
C: [3]
--------------------
Move disk 2 from B to C
A: [1]
B: []
C: [3, 2]
--------------------
Move disk 1 from A to C
A: []
B: []
C: [3, 2, 1]
--------------------


### BFS Solution

In [ ]:
from collections import deque
import csv
from itertools import permutations

class HanoiBFS:
    def __init__(self, n, start_rod, start_order, goal_rod, goal_order):
        self.n = n
        self.start_rod = start_rod
        self.start_order = start_order
        self.goal_rod = goal_rod
        self.goal_order = goal_order

        # Rod labels
        self.rods = ["A", "B", "C"]

        # Build initial and goal states
        start_stack = tuple(range(n, 0, -1)) if start_order == "decreasing" else tuple(range(1, n + 1))
        goal_stack = tuple(range(n, 0, -1)) if goal_order == "decreasing" else tuple(range(1, n + 1))

        self.start_state = tuple(
            start_stack if rod == self.start_rod else tuple() for rod in self.rods
        )
        self.goal_state = tuple(
            goal_stack if rod == self.goal_rod else tuple() for rod in self.rods
        )

        # Generate all possible actions
        self.actions = {}
        self._generate_actions()

    def _generate_actions(self):
        rule_no = 1
        for disk in range(1, self.n + 1):
            for from_rod, to_rod in permutations(self.rods, 2):
                self.actions[rule_no] = f"Move disk {disk} from {from_rod} to {to_rod}"
                rule_no += 1

    def _can_place(self, disk, rod_stack, order):
        if not rod_stack:
            return True
        top_disk = rod_stack[-1]
        if order == "decreasing":
            return disk < top_disk
        elif order == "increasing":
            return disk > top_disk
        return False

    def _get_order(self, rod):
        return self.start_order if rod == self.start_rod else (
            self.goal_order if rod == self.goal_rod else "decreasing"
        )

    def bfs(self):
        queue = deque([(self.start_state, [])])
        visited = {self.start_state}

        while queue:
            state, path = queue.popleft()

            if state == self.goal_state:
                return path

            for i in range(3):
                if not state[i]:
                    continue
                disk = state[i][-1]
                from_rod_label = self.rods[i]

                for j in range(3):
                    if i == j:
                        continue
                    to_rod_label = self.rods[j]
                    to_stack = state[j]

                    order = self._get_order(to_rod_label)
                    if self._can_place(disk, to_stack, order):
                        new_state = list(list(rod) for rod in state)
                        new_state[i] = new_state[i][:-1]
                        new_state[j] = new_state[j] + [disk]
                        new_state_tuple = tuple(tuple(rod) for rod in new_state)

                        if new_state_tuple not in visited:
                            visited.add(new_state_tuple)
                            rule_no = self._find_rule_no(disk, from_rod_label, to_rod_label)
                            queue.append((new_state_tuple, path + [(new_state_tuple, rule_no)]))
        return None

    def _find_rule_no(self, disk, from_rod, to_rod):
        for r, action in self.actions.items():
            if action == f"Move disk {disk} from {from_rod} to {to_rod}":
                return r
        return None

    def solve(self):
        path = self.bfs()
        if path is None:
            print("No solution found.")
            return

        # Print Action List
        print("\nAction List:")
        print(f"{'Rule no.':<8} {'Action'}")
        for r, action in self.actions.items():
            print(f"{r:<8} {action}")

        # Print Steps Table
        print("\nSteps Table:")
        print(f"{'S.No.':<6} {'Rod A':<15} {'Rod B':<15} {'Rod C':<15} {'Rule no.'}")
        step_no = 1
        print(f"{step_no:<6} {str(list(self.start_state[0])):<15} {str(list(self.start_state[1])):<15} {str(list(self.start_state[2])):<15} Initial state")
        step_no += 1
        for state, rule_no in path:
            print(f"{step_no:<6} {str(list(state[0])):<15} {str(list(state[1])):<15} {str(list(state[2])):<15} {rule_no}")
            step_no += 1

        # Export to CSV
        with open("action_list.csv", "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["Rule no.", "Action"])
            for r, action in self.actions.items():
                writer.writerow([r, action])

        with open("steps_table.csv", "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["S.No.", "Rod A", "Rod B", "Rod C", "Rule no."])
            step_no = 1
            writer.writerow([step_no, list(self.start_state[0]), list(self.start_state[1]), list(self.start_state[2]), "Initial state"])
            step_no += 1
            for state, rule_no in path:
                writer.writerow([step_no, list(state[0]), list(state[1]), list(state[2]), rule_no])
                step_no += 1

# ----------------- MAIN -----------------
if __name__ == "__main__":
    n = int(input("Enter number of discs: "))
    start_rod = input("Enter initial rod (A/B/C): ").upper()
    start_order = input("Enter initial arrangement (increasing/decreasing): ").lower()
    goal_rod = input("Enter final rod (A/B/C): ").upper()
    goal_order = input("Enter final arrangement (increasing/decreasing): ").lower()

    hanoi = HanoiBFS(n, start_rod, start_order, goal_rod, goal_order)
    hanoi.solve()


Enter number of discs: 3
Enter initial rod (A/B/C): A
Enter initial arrangement (increasing/decreasing): decreasing
Enter final rod (A/B/C): C
Enter final arrangement (increasing/decreasing): decreasing

Action List:
Rule no. Action
1        Move disk 1 from A to B
2        Move disk 1 from A to C
3        Move disk 1 from B to A
4        Move disk 1 from B to C
5        Move disk 1 from C to A
6        Move disk 1 from C to B
7        Move disk 2 from A to B
8        Move disk 2 from A to C
9        Move disk 2 from B to A
10       Move disk 2 from B to C
11       Move disk 2 from C to A
12       Move disk 2 from C to B
13       Move disk 3 from A to B
14       Move disk 3 from A to C
15       Move disk 3 from B to A
16       Move disk 3 from B to C
17       Move disk 3 from C to A
18       Move disk 3 from C to B

Steps Table:
S.No.  Rod A           Rod B           Rod C           Rule no.
1      [3, 2, 1]       []              []              Initial state
2      [3, 2]          [] 